# 高斯过程简介

在许多情况下，机器学习相当于从数据中估计参数。这些参数通常是众多且相对难以解释的——比如神经网络的权重。相比之下，高斯过程提供了一种直接推理可能适合我们数据的函数的高级属性的机制。例如，我们可能会对这些函数是否快速变化、周期性、涉及条件独立性或平移不变性有所了解。高斯过程使我们能够通过直接指定可能适合我们数据的函数值的高斯分布，轻松地将这些属性纳入我们的模型。

让我们通过一些例子来感受一下高斯过程是如何运作的。

假设我们观察到以下数据集，其中回归目标（输出）$y$ 由输入 $x$ 索引。例如，目标可能是二氧化碳浓度的变化，而输入可能是记录这些目标的时间。数据有哪些特征？它看起来变化有多快？我们是否有在规则间隔收集的数据点，还是有缺失的输入？你会如何想象填补缺失区域，或者预测直到 $x=25$？

![观测数据。](../img/gp-observed-data.svg)

为了用高斯过程拟合数据，我们首先指定了一个先验分布，表示我们认为哪些类型的函数是合理的。这里我们展示了来自高斯过程的一些样本函数。这个先验看起来合理吗？请注意，这里我们不是在寻找适合我们数据集的函数，而是为了指定解决方案的合理高级属性，例如它们随输入变化的速度。注意，我们将在下一个关于先验和推断的笔记本中看到重现本笔记本中所有图表的代码。

![我们希望用模型表示的样本先验函数。](../img/gp-sample-prior-functions.svg)

一旦我们基于数据进行条件化，我们可以使用这个先验来推断出可以拟合数据的函数的后验分布。这里我们展示了样本后验函数。

![一旦观察到数据后的样本后验函数。](../img/gp-sample-posterior-functions.svg)

我们看到这些函数中的每一个都与我们的数据完全一致，完美地穿过每个观测值。为了使用这些后验样本来进行预测，我们可以取后验中所有可能样本函数的值的平均值，以创建下面的粗蓝线。请注意，实际上我们不需要取无限数量的样本来计算这个期望；正如我们稍后会看到的，我们可以以封闭形式计算这个期望。

![后验样本，以及可用于点预测的后验均值，在蓝色中显示。](../img/gp-posterior-samples.svg)

我们可能还需要一种不确定性表示，以便我们知道应该对预测有多大的信心。直观上，在样本后验函数中有更多变异性的地方，我们应该有更多的不确定性，因为这告诉我们真实函数可能取很多不同的值。这种类型的不确定性称为_认识论不确定性_，它是由于信息不足而导致的_可减少的不确定性_。随着我们获取更多的数据，这种类型的不确定性会消失，因为会有越来越少的解与我们观察到的一致。就像后验均值一样，我们可以以封闭形式计算后验方差（后验中这些函数的变异性）。我们用阴影显示了均值两侧两倍的标准差，创建了一个95%的概率包含任何输入$x$的真实函数值的_可信区间_。

![后验样本，包括95%可信集。](../img/gp-posterior-samples-95.svg)

如果我们去掉后验样本，只可视化数据、后验均值和95%可信集，图看起来会更干净。注意，远离数据时不确定性增加，这是认识论不确定性的特性。

![点预测和可信集。](../img/gp-point-predictions.svg)

我们用来拟合数据的高斯过程的性质强烈受到所谓的_协方差函数_（也称为_核_）控制。我们使用的协方差函数被称为_RBF（径向基函数）核_，其形式为
$$ k_{\textrm{RBF}}(x,x') = \textrm{Cov}(f(x),f(x')) = a^2 \exp\left(-\frac{1}{2\ell^2}||x-x'||^2\right) $$

该核的_超参数_是可解释的。_幅度_参数$a$ 控制函数变化的垂直尺度，而_长度尺度_参数
$\ell$
控制函数的变化率（波纹度）。较大的$a$ 意味着较大的函数值，而较大的 
$\ell$ 
意味着变化较慢的函数。让我们看看当改变$a$ 和 
$\ell$ 时，我们的样本先验和后验函数会发生什么变化。

_长度尺度_对GP的预测和不确定性具有特别显著的影响。在 
$||x-x'|| = \ell$
时，一对函数值之间的协方差为$a^2\exp(-0.5)$。在大于 
$\ell$
的距离下，函数值几乎不相关。这意味着，如果我们想在某一点$x_*$ 进行预测，那么对于输入$x$ 使得 
$||x-x'||>\ell$
的函数值对我们的预测影响不大。

让我们看看改变长度尺度如何影响样本先验和后验函数以及可信集。上述拟合使用的长度尺度为$2$。现在考虑 
$\ell = 0.1, 0.5, 2, 5, 10$
。相对于我们正在考虑的输入域范围$25$，长度尺度为$0.1$ 是非常小的。例如，在这样的长度尺度下，$x=5$ 和 $x=10$ 处的函数值基本上没有相关性。另一方面，对于长度尺度为$10$，这些输入处的函数值高度相关。请注意，以下图形中的垂直尺度发生了变化。

![priorpoint1](../img/gp-priorpoint1.svg)
![postpoint1](../img/gp-postpoint1.svg)

![priorpoint5](../img/gp-priorpoint5.svg)
![postpoint5](../img/gp-postpoint5.svg)

![prior2](../img/gp-prior2.svg)
![post2](../img/gp-post2.svg)

![prior5](../img/gp-prior5.svg)
![post5](../img/gp-post5.svg)

注意到随着长度尺度的增加，函数的“波纹度”减小，我们的不确定性也随之减小。如果长度尺度较小，当我们远离数据时，不确定性会迅速增加，因为数据点对函数值的信息量减少。

现在，让我们固定长度尺度为$2$，并改变幅度参数。请注意，对于先验样本，垂直尺度保持固定，而后验样本则有所不同，因此您可以清楚地看到函数规模的增加以及数据拟合情况。

![priorap1](../img/gp-priorap1.svg)
![postapoint1](../img/gp-postapoint1.svg)

![priora2](../img/gp-priora2.svg)
![posta2](../img/gp-posta2.svg)

![priora8](../img/gp-priora8.svg)
![posta8](../img/gp-posta8.svg)

我们看到幅度参数影响函数的规模，但不影响变化率。此时，我们也感觉到我们的程序的泛化性能将取决于这些超参数的合理值。$\ell=2$ 和 $a=1$ 的值似乎提供了合理的拟合，而其他一些值则没有。幸运的是，有一种稳健且自动的方法来指定这些超参数，即使用所谓的_边际似然_，我们将在推断的笔记本中回到这一点。

那么，什么是高斯过程呢？正如我们开始所说的，高斯过程只是说任意一组函数值
$f(x_1),\dots,f(x_n)$，
由任意一组输入
$x_1,\dots,x_n$
索引，都有联合多变量高斯分布。这个分布的均值向量$\mu$ 由一个_均值函数_给出，通常被设为常数或零。这个分布的协方差矩阵由_核_在所有输入$x$ 对上的评估给出。

$$\begin{bmatrix}f(x) \\f(x_1) \\ \vdots \\ f(x_n) \end{bmatrix}\sim \mathcal{N}\left(\mu, \begin{bmatrix}k(x,x) & k(x, x_1) & \dots & k(x,x_n) \\ k(x_1,x) & k(x_1,x_1) & \dots & k(x_1,x_n) \\ \vdots & \vdots & \ddots & \vdots \\ k(x_n, x) & k(x_n, x_1) & \dots & k(x_n,x_n) \end{bmatrix}\right)$$
:eqlabel:`eq_gp_prior`

公式 :eqref:`eq_gp_prior` 指定了一个GP先验。我们可以计算给定$f(x_1), \dots, f(x_n)$，即我们已经观察到的函数值，任何$x$ 处的$f(x)$ 的条件分布。这个条件分布称为_后验_，是我们用来进行预测的。

特别是，

$$f(x) | f(x_1), \dots, f(x_n) \sim \mathcal{N}(m,s^2)$$ 

其中

$$m = k(x,x_{1:n}) k(x_{1:n},x_{1:n})^{-1} f(x_{1:n})$$ 

$$s^2 = k(x,x) - k(x,x_{1:n})k(x_{1:n},x_{1:n})^{-1}k(x,x_{1:n})$$ 

其中$k(x,x_{1:n})$ 是通过评估$k(x,x_{i})$ 对于$i=1,\dots,n$ 形成的$1 \times n$ 向量，$k(x_{1:n},x_{1:n})$ 是通过评估$k(x_i,x_j)$ 对于$i,j = 1,\dots,n$ 形成的$n \times n$ 矩阵。$m$ 是我们可以用作任何$x$ 的点预测器，而$s^2$ 是我们用于不确定性的：如果我们想要创建一个有95%概率包含$f(x)$ 的区间，我们将使用$m \pm 2s$。以上所有图中的预测均值和不确定性都是使用这些公式创建的。观察到的数据点由
$f(x_1), \dots, f(x_n)$
给出，并选择了一组精细的$x$ 点来进行预测。

假设我们观察到单个数据点$f(x_1)$，并且我们想确定某个$x$ 处的$f(x)$ 值。因为$f(x)$ 由高斯过程描述，我们知道
$(f(x), f(x_1))$
的联合分布是高斯的：

$$
\begin{bmatrix}
f(x) \\ 
f(x_1) \\
\end{bmatrix}
\sim
\mathcal{N}\left(\mu, 
\begin{bmatrix}
k(x,x) & k(x, x_1) \\
k(x_1,x) & k(x_1,x_1)
\end{bmatrix}
\right)
$$

非对角表达式$k(x,x_1) = k(x_1,x)$
告诉我们函数值的相关程度——$f(x)$
将由$f(x_1)$ 强烈决定的程度。
我们已经看到，如果我们使用相对于$x$ 和$x_1$ 之间的距离$||x-x_1||$ 较大的长度尺度，则函数值将高度相关。我们可以在函数空间和$f(x_1), f(x)$ 的联合分布中可视化确定$f(x)$ 从$f(x_1)$ 的过程。让我们最初考虑一个$x$，使得$k(x,x_1) = 0.9$，且$k(x,x)=1$，这意味着$f(x)$ 的值与$f(x_1)$ 的值适度相关。在联合分布中，恒定概率的轮廓将是相对较窄的椭圆。

假设我们观察到$f(x_1) = 1.2$。
为了根据这个$f(x_1)$ 的值进行条件化，
我们可以在密度图上画一条水平线在$1.2$，并看到$f(x)$
的值主要被限制在$[0.64,1.52]$ 内。我们还在函数空间中绘制了这张图，显示了观察到的
点$f(x_1)$ 为橙色，以及$f(x)$ 的高斯过程预测分布的一个标准差为蓝色，均值为$1.08$。

![当$k(x,x_1) = 0.9$ 时，$f(x_1)$ 和 $f(x)$ 的双变量高斯密度的等概率轮廓。](https://user-images.githubusercontent.com/6753639/206867364-b4707db5-0c2e-4ae4-a412-8292bca4d08d.svg)
![当$k(x,x_1) = 0.9$ 时，在函数空间中$f(x)$ 的高斯过程预测分布。](https://user-images.githubusercontent.com/6753639/206867367-3815720c-93c8-4b4b-80e7-296db1d3553b.svg)

现在假设我们有更强的相关性，$k(x,x_1) = 0.95$。
现在椭圆变得更窄了，$f(x)$
的值甚至更强烈地由$f(x_1)$ 决定。在$1.2$ 处画一条水平线，我们看到$f(x)$
的轮廓支持值主要在$[0.83, 1.45]$ 内。同样，我们在函数空间中也显示了这张图，有一个标准差围绕预测均值$1.14$。

![当$k(x,x_1) = 0.95$ 时，$f(x_1)$ 和 $f(x)$ 的双变量高斯密度的等概率轮廓。](https://user-images.githubusercontent.com/6753639/206867797-20e42783-31de-4c50-8103-e9441ba6d0a9.svg)
![当$k(x,x_1)$ = 0.95 时，在函数空间中$f(x)$ 的高斯过程预测分布。](https://user-images.githubusercontent.com/6753639/206867800-d9fc7add-649d-492c-8848-cab07c8fb83e.svg)

我们看到，我们的高斯过程后验均值预测器更接近$1.2$，因为现在有更强的相关性。我们还看到，我们的不确定性（误差条）有所减少。尽管这些函数值之间有很强的相关性，但由于我们只观察到了一个数据点，所以我们的不确定性仍然相当大！

此过程可以为我们提供任何$x$ 处的$f(x)$ 的后验，无论我们观察了多少点。假设我们观察到$f(x_1), f(x_2)$。我们现在在函数空间中可视化特定$x=x'$ 处的$f(x)$ 的后验。$f(x)$ 的精确分布由上述方程给出。$f(x)$ 是高斯分布的，均值为

$$m = k(x,x_{1:3}) k(x_{1:3},x_{1:3})^{-1} f(x_{1:3})$$

方差为

$$s^2 = k(x,x) - k(x,x_{1:3})k(x_{1:3},x_{1:3})^{-1}k(x,x_{1:3})$$

在这个入门笔记本中，我们一直考虑的是_无噪声_ 观测。正如我们将看到的，很容易包括观测噪声。如果我们假设数据是由潜在的无噪声函数$f(x)$ 加上iid高斯噪声
$\epsilon(x) \sim \mathcal{N}(0,\sigma^2)$
生成的，方差为$\sigma^2$，那么我们的协方差函数简单地变为
$k(x_i,x_j) \to k(x_i,x_j) + \delta_{ij}\sigma^2$，
其中$\delta_{ij} = 1$ 如果$i=j$，否则为$0$。

我们已经开始对如何使用高斯过程来指定先验和后验解，以及核函数如何影响这些解的属性有了直观的认识。在接下来的笔记本中，我们将精确展示如何指定高斯过程先验，介绍并推导各种核函数，然后详细介绍如何自动学习核超参数，形成高斯过程后验以进行预测。虽然需要时间和实践来习惯诸如“函数分布”这样的概念，但实际上找到GP预测方程的机制其实相当简单——这使得容易获得实践并形成对这些概念的直观理解。

## 总结

在典型的机器学习中，我们指定一个带有一些自由参数的函数（如神经网络及其权重），我们专注于估计这些参数，这些参数可能不可解释。使用高斯过程，我们直接推理函数的分布，这使我们能够推理解决方案的高级属性。这些属性由协方差函数（核）控制，通常有几个高度可解释的超参数。这些超参数包括_长度尺度_，它控制函数变化的速度（波纹度）。另一个超参数是幅度，它控制函数变化的垂直尺度。
表示许多可以拟合数据的不同函数，并将它们组合成一个预测分布，是贝叶斯方法的显著特征。因为在远离数据的地方，可能的解之间的差异更大，我们的不确定性直观上随着我们远离数据而增加。

高斯过程通过指定所有可能函数值的多元正态（高斯）分布来表示函数的分布。可以轻松操作高斯分布，以找到基于其他任何一组值的函数值的分布。换句话说，如果我们观察到一组点，那么我们可以基于这些点进行条件化，并推断函数在任何其他输入下的值可能是什么样的分布。我们如何建模这些点之间的相关性由协方差函数决定，并定义了高斯过程的泛化属性。虽然需要时间来适应高斯过程，但它们易于处理，有许多应用，并帮助我们理解和开发其他模型类，如神经网络。

## 练习

1. 认识论不确定性与观测不确定性有什么区别？
2. 除了变化率和幅度外，我们还可能考虑函数的哪些其他属性，以及这些属性在现实世界中的函数示例是什么？
3. 我们考虑的RBF协方差函数表明，观测之间的协方差（和相关性）随着输入空间（时间、空间位置等）中距离的增加而减小。这是一个合理的假设吗？为什么？
4. 两个高斯变量的和是高斯的吗？两个高斯变量的乘积是高斯的吗？如果(a,b)具有联合高斯分布，那么a|b（给定b的a）是高斯的吗？a是高斯的吗？
5. 重复练习，其中我们在$f(x_1) = 1.2$ 处观察到一个数据点，但现在假设我们另外观察到$f(x_2) = 1.4$。令$k(x,x_1) = 0.9$，且$k(x,x_2) = 0.8$。相比于仅观察到$f(x_1)$ 时，我们对$f(x)$ 的值会更加确定还是更不确定？现在$f(x)$ 的均值和95%可信集是多少？
6. 你认为增加我们对观测噪声的估计会增加还是减少我们对地面真值函数长度尺度的估计？
7. 当我们远离数据时，假设我们预测分布的不确定性增加到一定程度，然后停止增加。为什么会发生这种情况？

[讨论](https://discuss.d2l.ai/t/12115)